In [2]:
%reload_ext autoreload
%autoreload 2

In [15]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
from Scratch import metadata_loader
provider = SaverProvider()

In [35]:
df = metadata_loader(extract=True)
#df = df[df.job_status != 'JobStatus.QUEUED']


In [36]:
try:
    for job_id in tqdm(df.job_id):
        provider.retrieve_job(job_id, _ignore_running=True)
except KeyboardInterrupt:
    print('Interrupted')

  0%|          | 0/233 [00:00<?, ?it/s]

18:32:12 Warning: Job ID cmz99exrmwhg008bt1f0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
18:32:30 Warning: Job ID cmz99exrmwhg008bt1f0 is still running. Aborting.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 86
18:32:30 Warning: Job ID cmz99cxdaqbg008shx8g not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
18:32:47 Warning: Jo

In [ ]:
df = df[df['job_id']=="cmxnz9dwjay0008cnwc0"]
if df.job_status.values == 'JobStatus.DONE':
    print("Job is done")    